# 1 Discriminator

In [ ]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add
from tensorflow.keras.layers import MaxPooling2D, UpSampling2D, Conv2D, Conv2DTranspose
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam
import tensorflow as tf
from random import randint
 
from google.colab.patches import cv2_imshow
import numpy as np
from google.colab import drive
 
from tensorflow.python.framework.ops import disable_eager_execution, enable_eager_execution

import random

from tensorflow.keras.applications.vgg19 import VGG19
import glob

In [ ]:
import logging
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
MODELS_PATH = "/content/gdrive/MyDrive/Projects/GDL Images/models/MyGAN/"
D_PATH = MODELS_PATH+"disc.h5"
G_PATH = MODELS_PATH+"generator.h5"

In [ ]:
drive.mount('/content/gdrive/')

In [ ]:
!ln -s "/content/gdrive/MyDrive/Projects/GDL Images/dataset/monet.npy" "/content/monet.npy"
!ln -s "/content/gdrive/MyDrive/Projects/GDL Images/dataset/not/" "/content/"

In [ ]:
def loadNpy(path):
  return np.array(np.load(path, mmap_mode='r')/255,dtype=np.float64)

In [ ]:
class DataRemember():
  def __init__(self, maxsize=1000):
    self.maxsize = maxsize
    self.array = []
  
  def AddToArray(self, add):
    free_size = self.maxsize - len(self.array)
    to_remove = len(add) - free_size
    if to_remove > 0:
      self.array = self.array[to_remove:]
    for i in add:
      self.array.append(i)
    random.shuffle(self.array)
  
  def ToNumpy(self):
    return np.array(self.array)

In [ ]:
def buildGeneratorVGG19(input_shape, weight_init):
  vgg19 = VGG19(weights='imagenet',include_top=False)
  def downSample(input_layer,filters,amount,max_pool=True):
    y = input_layer
    for _ in range(amount):
      y = Conv2D(filters,kernel_size=3, strides=1, padding='same', activation="relu")(y)
    if max_pool:
      y = MaxPooling2D(pool_size=2)(y)
    return y
  def upSample(input_layer,filters,amount,weight_init):
    y = UpSampling2D(size=(2,2))(input_layer)
    for _ in range(amount):
      y = Conv2D(filters,kernel_size=3, strides=1, padding='same', activation="relu",kernel_initializer=weight_init)(y)
      y = BatchNormalization()(y)
      y = Dropout(0.08)(y)
    return y
  input_layer = Input(shape=input_shape)
  block1_out = downSample(input_layer,64,2)
  block2_out = downSample(block1_out,128,2)
  block3_out = downSample(block2_out,256,4)
  block4_out = downSample(block3_out,512,4)
  block5_out = downSample(block4_out,512,4,False)
  y = Concatenate()([block5_out,block4_out])
  y = upSample(y,512,4,weight_init)
  y = Concatenate()([block3_out,y])
  y = upSample(y,256,4,weight_init)
  y = Concatenate()([block2_out,y])
  y = upSample(y,128,2,weight_init)
  y = Concatenate()([block1_out,y])
  y = upSample(y,64,2,weight_init)
  output = Conv2D(3,kernel_size=3, strides=1, padding='same', activation="sigmoid",kernel_initializer=weight_init)(y)

  model = Model(input_layer,output)

  vgg19_layers = vgg19.layers
  model_layers = model.layers
  i = 0
  while i<len(vgg19_layers)-1:
    model_layers[i].set_weights(vgg19_layers[i].get_weights())
    model_layers[i].trainable = False
    i+=1
  return model

In [ ]:
def buildDiscriminator(input_shape,weight_init):
  def conv4(layer_input,filters,weight_init,norm=True,stride=2):
    y = Conv2D(filters, kernel_size=4, strides=stride, padding='same', kernel_initializer = weight_init, activation="relu")(layer_input)
    if norm:
      y = BatchNormalization()(y)
    return y

  img = Input(shape=input_shape)

  y = conv4(img, 8, weight_init, norm = False)
  y = conv4(y, 16, weight_init)
  y = MaxPooling2D()(y)
  y = conv4(y, 32, weight_init)
  y = MaxPooling2D()(y)
  output = Conv2D(1, kernel_size=2, strides=2, padding='same',kernel_initializer = weight_init, activation="sigmoid")(y)

  return Model(img, output)

In [ ]:
class MyGAN():
  def __init__(self, input_dim, disc_path, gen_path, load_models=False):
    weight_init = RandomNormal(mean=0., stddev=0.02)
    self.input_shape = input_dim
    if not load_models:
      self.generator = buildGeneratorVGG19(self.input_shape, weight_init)
    else:
      self.generator = tf.keras.models.load_model(gen_path)
    self.generator.trainable = True
    if not load_models:
      self.discriminator = buildDiscriminator(self.input_shape, weight_init)
    else:
      self.discriminator = tf.keras.models.load_model(disc_path)
    
    self.d_path = disc_path
    self.g_path = gen_path
    self.queue = DataRemember()
    output_shape_disc = self.discriminator.output.shape
    self.patch = output_shape_disc[1:]
    self.CompileModels()

  def CompileModels(self):
    self.discriminator.compile(
        loss="mse",
        optimizer=Adam(),
        metrics=['accuracy']
    )
    img = Input(shape=self.input_shape)
    generated = self.generator(img)
    output = self.discriminator(generated)

    self.combined = Model(img,output)
    self.combined.compile(
        loss="mse",
        optimizer=SGD(),
        metrics=['accuracy']
    )
  
  def TrainDiscriminator(self):
    self.discriminator.trainable = True
    print("Training discriminator...")
    print(" Generating data...")
    generated = self.generator.predict(self.data_B)
    self.queue.AddToArray(generated)
    generated = self.queue.ToNumpy()
    print(" Finished generating data...")
    zeros = np.zeros((generated.shape[0]+self.data_B.shape[0],)+self.patch)
    x = np.concatenate((self.data_A,generated,self.data_B))
    y = np.concatenate((self.disc_ones,zeros))
    self.discriminator.fit(x,y,shuffle=True,epochs=5,batch_size=32)

  def TrainGenerator(self):
    self.discriminator.trainable = False
    print("Training generator...")
    y = np.ones((self.data_B.shape[0],)+self.patch)
    x = self.data_B
    self.combined.fit(x,y,epochs=1,batch_size=4)

  def ShowProgress(self):
    i = random.randint(0, self.data_B.shape[0])
    img = np.array([self.data_B[i]])
    generated = np.array(self.generator.predict(img)[0]*255,dtype=np.int64)
    img = img[0]*255
    cv2_imshow(img)
    cv2_imshow(generated)

  def saveModels(self):
    self.discriminator.trainable = True
    self.discriminator.save(self.d_path)
    self.generator.save(self.g_path)
    print("saved")

  def train(self, path_A, path_B, times=5):
    files = glob.glob(path_B)
    random.shuffle(files)
    self.data_A = loadNpy(path_A)
    self.disc_ones = np.ones((self.data_A.shape[0],)+self.patch)
    for i in range(1,times+1):
      print("{0}/{1}".format(i,times))
      for B_file in files:
        self.data_B = loadNpy(B_file)
        self.TrainDiscriminator()
        self.TrainGenerator()
        self.ShowProgress()
        self.saveModels()

In [ ]:
model = MyGAN((256,256,3),D_PATH,G_PATH,True)

In [ ]:
model.train("monet.npy","/content/not/*.npy")